##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Normalizações

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/layers_normalizations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/addons/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


## Visão geral

Este notebook fornece uma breve introdução às [camadas de normalização](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py) do TensorFlow. As camadas compatíveis no momento são:

- **Normalização de grupo** (TensorFlow Addons)
- **Normalização de instância** (TensorFlow Addons)
- **Normalização de camada** (TensorFlow Core)

A ideia básica por trás dessas camadas é normalizar a saída de uma camada de ativação para melhorar a convergência durante o treinamento. Em comparação com a [normalização de lote](https://keras.io/layers/normalization/), essas normalizações não funcionam em lotes. Em vez disso, elas normalizam as ativações de uma única amostra, tornando-as adequadas para redes neurais recorrentes também.

Geralmente, a normalização é realizada ao calcular a média e o desvio padrão de um subgrupo no seu tensor de entrada. Também é possível aplicar uma escala e um fator de deslocamento.

$y_{i} = \frac{\gamma ( x_{i} - \mu )}{\sigma }+ \beta$

$ y$ : saída

$x$ : entrada

$\gamma$ : fator de escala

$\mu$: média

$\sigma$: desvio padrão

$\beta$: fator de deslocamento

A seguinte imagem demonstra a diferença entre essas técnicas. Cada subplot mostra um tensor de entrada, com N como o eixo do lote, C como o eixo do canal e (H, W) como os eixos espaciais (a altura e largura de uma foto, por exemplo). Os pixels em azul são normalizados pela mesma média e variância, computadas ao agregar os valores desses pixels.

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

Fonte: (https://arxiv.org/pdf/1803.08494.pdf)

Os pesos gamma e beta são treináveis em todas as camadas de normalização para compensar a possível perda de capacidade representacional. Você pode ativar esses fatores ao definir a sinalização `center` ou `scale` como `True`. É claro que você pode usar `initializers`, `constraints` e `regularizer` para `beta` e `gamma`, ajustando esses valores durante o processo de treinamento. 

## Configuração

### Instale o Tensorflow 2.0 e o Tensorflow-Addons

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

### Preparando o dataset

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Tutorial de normalização de grupo

### Introdução

A normalização de grupo (GN) divide os canais das suas entradas em subgrupos menores e normaliza esses valores com base na média e na variância. Como a GN funciona em um único exemplo, essa técnica não depende do tamanho do lote.

A GN teve uma pontuação experimentalmente mais próxima à normalização de lotes em tarefas de classificação de imagens. Pode ser vantajoso usar a GN em vez da normalização de lotes caso seu batch_size geral seja baixo, o que levaria ao mau desempenho da normalização de lotes

###Exemplo Divisão de 10 canais após uma camada Conv2D em 5 subgrupos em uma configuração "channels last" padrão:

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalização de instância

### Introdução

A normalização de instância é o caso especial de normalização de grupo em que o tamanho do grupo é igual ao tamanho do canal (ou do eixo).

Os resultados experimentais mostram que a normalização de instância tem um bom desempenho na transferência de estilo ao substituir a normalização de lote. Recentemente, a normalização de instância também foi usada como substituta para a normalização de lote em GANs.

### Exemplo

Aplicando InstanceNormalization após uma camada Conv2D e usando uma escala inicializada uniforme e um fator de deslocamento.

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis=3, 
                                   center=True, 
                                   scale=True,
                                   beta_initializer="random_uniform",
                                   gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalização de camada

### Introdução

A normalização de camada é um caso especial de normalização de grupo em que o tamanho do grupo é 1. A média e o desvio padrão são calculados de todas as ativações de um único exemplo.

Os resultados experimentais mostram que a normalização de camada é bastante adequada para redes neurais recorrentes, já que trabalha com o tamanho do lote de maneira independente.

### Exemplo

Aplicando Layernormalization após uma camada Conv2D Layer e usando uma escala e um fator de deslocamento. 

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=3 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Bibliografia

[Normalização de camada](https://arxiv.org/pdf/1607.06450.pdf)

[Normalização de instância](https://arxiv.org/pdf/1607.08022.pdf)

[Normalização de grupo](https://arxiv.org/pdf/1803.08494.pdf)